In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pyperclip
import requests
from threading import Thread
from functions import *

/Users/enriccogemha/Developer/PokemonTCG-Card-Network/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
URL = 'https://limitlesstcg.com'


In [3]:
# Initialize the WebDriver (in this case, for Chrome)
driver = webdriver.Chrome()

filters = ['regional', 'national', 'international', 'worlds', 'special', 'cl', 'rl', 'online', 'players_cup', 'invitational']
tournament_links_by_type = {'regional': [], 'national': [], 'international': [], 'worlds': [], 'others': []}

for filter in filters:
    # Go to URL for all tournaments between 2011 and 2023
    driver.get(URL+f'/tournaments?time=all&type={filter}&format=all&region=all')

    # Get all the links to the tournaments
    tls = get_all_tournaments(driver)
    if filter in ['regional', 'national', 'international', 'worlds']:
        tournament_links_by_type[filter] = tls
    else:
        tournament_links_by_type['others'] += tls

driver.close() # Let's not the same session open for too long

print(f"Regionals: {len(tournament_links_by_type['regional'])}")
print(f"Nationals: {len(tournament_links_by_type['national'])}")
print(f"Internationals: {len(tournament_links_by_type['international'])}")
print(f"Worlds: {len(tournament_links_by_type['worlds'])}")
print(f"Others: {len(tournament_links_by_type['others'])}")

The chromedriver version (117.0.5938.92) detected in PATH at /opt/homebrew/bin/chromedriver might not be compatible with the detected chrome version (118.0.5993.70); currently, chromedriver 118.0.5993.70 is recommended for chrome 118.*, so it is advised to delete the driver in PATH and retry


Regionals: 25
Nationals: 23
Internationals: 20
Worlds: 11
Others: 68


In [4]:
with open('data/tournaments.csv', 'w') as f:
        f.write('id_card,name_card,amount_card,price_card,energy_type_card,type_card,combo_type_id,combo_type_name,id_player,name_player,country_player,all_time_score,ranking_player_tournament,id_tournament,category_tournament,name_tournament,country_tournament,year_tournament,month_tournament,day_tournament,valid_rotation_at_tournament\n')

In [6]:
# Initialize the WebDriver (in this case, for Chrome)
driver = webdriver.Chrome()

# Iterate through all the categories
for category, tournament_links in tournament_links_by_type.items():
    
    # Iterate through all the tournaments
    for tl in tournament_links:
        
        # Go to the tournament page
        driver.get(tl)

        # Call the function to get the data from a tournament
        tournament_data = get_tournament_data(driver, category, int(tl.split('/')[-1]))

        # Close the session
        driver.close()


The chromedriver version (117.0.5938.92) detected in PATH at /opt/homebrew/bin/chromedriver might not be compatible with the detected chrome version (118.0.5993.70); currently, chromedriver 118.0.5993.70 is recommended for chrome 118.*, so it is advised to delete the driver in PATH and retry


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[2]/div/div[1]/div[1]/img"}
  (Session info: chrome=118.0.5993.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000100526d98 chromedriver + 4337048
1   chromedriver                        0x000000010051ee14 chromedriver + 4304404
2   chromedriver                        0x000000010014ba5c chromedriver + 293468
3   chromedriver                        0x0000000100190d50 chromedriver + 576848
4   chromedriver                        0x00000001001cb908 chromedriver + 817416
5   chromedriver                        0x0000000100184a5c chromedriver + 526940
6   chromedriver                        0x0000000100185908 chromedriver + 530696
7   chromedriver                        0x00000001004ecde4 chromedriver + 4099556
8   